## Pipeline

The following pipeline calls other scripts, which will load their own libraries as required.

The pipeline loads the output from SpaceRanger (the count matrix) and combines your data and analysis into a Seurat Object. 

Before being able to create visualizations that may be shown in the app, your SpaceRanger output must be run through the following steps:

### Steps
0. Loading the Spatial Datasets
1. Inspecting the Spatial Datasets
2. Quality control of the Spatial Data: Filtering number of Genes and UMIs
3. Merging into one Seurat Object
4. Normalisation
5. Getting the Pearson Correlation Coefficient
6. Finding Highly Variable Features
7. Identification of Spatially Variable Features
8. Generating the Elbow Plot
9. Clustering: UMAPs and Spatial Plots
10. Finding Marker Genes
11. Saving the processed Seurat object into an RDS file
   
Each step has variables that you will have to change according to your samples/organisms.

### Setup

#### Load the necessary libraries

In [ ]:
library(Seurat)
library(data.table)

#### Set the seed value

In [ ]:
SEED <- 314
set.seed(SEED)

Set the name for this run. 

Change this as needed; changing it will not affect the calculations, only the name of the folder where the output is written to.

In [ ]:
run_name <- "______"

#### Specify the input and output folders. 

The default folders are used, but you may have started the Docker container with a different volume mounting path for your data, in which case you need to specify it here instead of the default.

The 'input_data_folder' must contain 2 files (outputs of SpaceRanger):
- the filtered_feature_bc_matrix.h5 file 
- features.tsv.gz (THIS MIGHT NOT BE NEEDED HERE)

The output_folder is where all pipeline outputs will go

In [ ]:
input_folder <- "/tmp/work/" 
output_folder <- paste0("/tmp/work/pipeline/", run_name, "/")
data_folder <- paste0(input_folder, "/data")

#### Variables

Feel free to change the value of the following variables

- <b> name </b> - name of your dataset
- <b> sections </b>- capture areas in the Visium slides
- <b> timepoints </b>- timepoint you'd like to process. May be a list of all the possible timepoints
- <b> levels </b> - timepoint and Slide section

In [ ]:
# User dependent
name <- "0HAI"
sections <- c("A", "B", "C", "D")
timepoints <- c("0HAI")
levels <- c("0HAI_A", "0HAI_B", "0HAI_C", "0HAI_D")

Variables for quality control (filtration of the data) and plotting.
- <b> alpha </b>  - Statitical significance, used in 'inspect data', 'filterdatagraphs', and 'inspectdatamerged'
- <b> color_red </b>  - hex value for red used in 'filterdatagraphs
- <b> color_green </b>  - hex value for green used in 'filterdatagraphs'

In [ ]:

alpha <- 0.75
color_red <- "#DF2E38"
color_green <- "#A7DF97"

### Step 0 - 3: Pre-processing of the spatial data

Set Variables: 

- <b> cut_off_nUMIs_list </b>- vectors of Unique Molecular Identifiers (UMI) to be used as filters in 02_Quality_control
- <b> cut_off_nGenes_list </b>- vectors of Numbers of expressed genes to be used as filters in 02_Quality_control
- <b> cut_off_nUMIs </b>- Cut off value to be applied to the merged seurat object
- <b> cut_off_nGenes </b>- Cut off value to be applied to the merged seurat object
- <b> seu_obj_unfiltered_list AND seu_obj_filtered_list </b>- lists that will contain filtered and unfiltered Seurat Objects

- <b> filtered_feature_bc_matrix_path </b> - Space Ranger output. Rename if necessary.
- <b>feature_tsv_file_path </b> - your own filtered_feature_bc_matrix. Rename if necessary.

In [ ]:

cut_off_nUMIs_list <- c(20, 20, 30, 30, 30, 100, 150, 200, 200, 200, 300, 300)
cut_off_nGenes_list <- c(10, 20, 10, 20, 30, 100, 150, 150, 200, 250, 250, 300)

cut_off_nUMIs <- list("0HAI" = 30)
cut_off_nGenes <- list("0HAI" = 10)

seu_obj_unfiltered_list <- NULL
seu_obj_filtered_list <- NULL

filtered_feature_bc_matrix_path <- "./filtered_feature_bc_matrix.h5"
feature_tsv_file_path <- paste0(data_folder, "/filtered_feature_bc_matrix/features.tsv.gz")

These next 3 steps inspects the data, filters them according to cut_off_nGenes and cut_off_nUMIs.

It then merges the data into one merged Seurat Object.
  
We iterate through the specified timepoints and sections

Each timepoint may have 4-8 sections

<b>Step 0</b>: loads the data in each combination of timepoints and sections

<b>Step 1</b>: inspects the data 
- It draws plots to determine the number of UMIs and number of genes expressed in each dataset.
  
<b>Step 2</b>: Filtering
- It takes the cut_off_nUMIs_list, cut_off_nGenes_list, applies each of those values, and produces plots of the results to help inform the value you want for 'cut_offnGenes' and 'cut_off_nUMIs'

<b>Step 3</b>: merges all the Unfiltered sections for inspection

In [ ]:
for(timepoint in timepoints) {
  for(section in sections) {
    dataset_name <- paste0(timepoint, "_", section)
    
    # STEP 0 - Load the data
    message(paste0("~ SCRIPT: Loading the data for: ", dataset_name, "..."))
    
    seu_obj <- Load10X_Spatial(
      data.dir = data_folder, 
      filename = filtered_feature_bc_matrix_path,
      assay = "Spatial",
      slice = dataset_name, 
      filter.matrix = TRUE, 
    )
    
    seu_obj <- RenameCells(object = seu_obj, add.cell.id = dataset_name)
    seu_obj@meta.data$section <- section
    seu_obj@meta.data$timepoint <- timepoint
    seu_obj@meta.data$orig.ident <- dataset_name
    seu_obj$section <- dataset_name
    Idents(seu_obj) <- dataset_name
      
    feature_df_data <- fread(feature_tsv_file_path, header=FALSE, sep='\t')
    seu_obj@assays[["Spatial"]]@counts@Dimnames[[1]] <- feature_df_data$V1
    seu_obj@assays[["Spatial"]]@data@Dimnames[[1]] <- feature_df_data$V1
    
    # STEP 1 - INSPECT DATA
    message(paste0("~ SCRIPT: Inspecting the data for: ", dataset_name, "..."))
    source("./01_Inspect_data.R")
    inspect_data(seu_obj, output_folder, dataset_name, alpha)
    
    seu_obj_unfiltered_list <- append(seu_obj_unfiltered_list, seu_obj)
    
    # STEP 2 - FILTER DATA
    message(paste0("~ SCRIPT: Filtering the data for: ", dataset_name, "..."))
    source("./02_Quality_control_(Filter_out).R")
    filter_data_graphs(seu_obj, output_folder, cut_off_nUMIs_list, cut_off_nGenes_list, 
                       alpha, color_red, color_green, dataset_name)
    seu_obj <- subset(seu_obj, subset = nFeature_Spatial >= cut_off_nGenes & nCount_Spatial >= cut_off_nUMIs)  
    seu_obj_filtered_list <- append(seu_obj_filtered_list, seu_obj)
  }
  
  # STEP 3 - INSPECT MERGED UNFILTERED DATA
  message(paste0("~ SCRIPT:Inspect the merged unfiltered data for: ", dataset_name, "..."))
  seu_obj_unfiltered_merged <- Reduce(merge, seu_obj_unfiltered_list)
  inspect_data_merged(seu_obj_unfiltered_merged, output_folder, name, alpha)
}


We now merge the filtered dataset

In [ ]:
seu_obj_merged <- Reduce(merge, seu_obj_filtered_list)

### Step 4: Normalisation

For each time point, the data from the sections were merged with the “merge” function from Seurat package (v. 4.1.0) and normalized using SCTransform (settings: variable.features.n = NULL, variable.features.rv.th = 1). 

Section variability within the data were regressed out using the ‘vars.to.regress’ function from SCTransform

- <b> regress_out </b> - If set to TRUE, it runs SCTransform with the parameters necessary to regress out unwanted variablility from the variable 'section'. 

In [ ]:
regress_out <- TRUE

In [ ]:
if(regress_out) {
  seu_obj_merged <- SCTransform(seu_obj_merged, assay = "Spatial", vars.to.regress =  c('section'),
                      variable.features.n = NULL, variable.features.rv.th = 1, return.only.var.genes = FALSE)
} else { 
  seu_obj_merged <- SCTransform(seu_obj_merged, assay = "Spatial", variable.features.n = NULL, return.only.var.genes = FALSE)
}

### Step 5: Correlation Coefficient

We calculate the correlation coefficient to gauge similarity between the each timepoint's sections.


In [ ]:
source("./05_Pearson.R")
calculate_Pearson_correlation(seu_obj_merged, output_folder, timepoints, sections)

### Step 6: Identification of highly variable features
This calculates a subset of genes that present high spot-to-spot variation in the dataset.

This step looks at which genes change the most in the dataset.


In [ ]:
source("./06_Highly_variable_features.R")
find_highly_variable_features(seu_obj_merged, output_folder, name)

### Step 7: Identification of Spatially Variable Features

It identifies genes that have spatial variability in their expression patterns, indicating that their expression levels differ significantly across different regions of the tissue.

In [ ]:
find_spatially_variable_feature_after_norm(seu_obj_merged, output_folder, name)

### Step 8: Create Elbow Plot 

This decides the number of dimensions to be used in analysis.

The value where the plot forms the elbow informs the value of the 'dims' variable.


In [ ]:
dims <- 30

In [ ]:
seu_obj_merged <- RunPCA(seu_obj_merged, assay = "SCT", npcs = 2*dims)

In [ ]:
source("./07_Elbow.R")
draw_elbow_plot(seu_obj_merged, name, output_folder, dims)

### Step 9: Clustering of the Spatial Data

Principal component analysis (PCA) 
was performed on the genes with residual variance > 1, and 
the 30 most significant components were retained as input for Uniform Manifold Approximation and Projection 
(UMAP) and for spot clustering.

- <b>assay</b> - "SCT" specifies that the PCA should be run using the data from the SCT assay.
- <b> dims </b> - Used in the variable 'npcs' of function 'RunPCA' and in Step 8: Clustering. The recommended value for 'dims' is informed by the Elbow plot.
- <b>npcs</b> - 2*dims specifies the number of principal components to calculate

The PCA will therefore be calculated using the scaled data from the SCT assay, which is stored in the @scale.data slot of the assay.

Generation of spot 
clusters was made using the ‘FindNeighbors’ (settings: reduction = ‘pca’, dims = 1:30) followed by the ‘FindClusters’ 
functions in Seurat, where the default algorithm was used 
to construct a Shared Nearest Neighbor (SSN) graph and 
apply the Louvain algorithm for cluster generation. 

Cluster resolution was set from 0.4 to 0.8. The identity of the clusters was assigned based on their location
in the section according to the morphology of a barley grain. 
To identify marker genes for each cluster, pair-wise comparisons of individual clusters 
against all other clusters were performed using the ‘FindAllMarkers’ function (settings: min.pct = 0.05, logfc.threshold = 0.25, only.pos = T) with 
Wilcox on rank sum test in the Seurat package. 
The marker genes were further filtered with an adjusted P value < 0.05 

This creates Spatial Feature Plots and UMAPs that show the clusters found.

Variables:

- <b> umap_clusters </b> - If True, runs the function 'RunUMAP'

- <b> resolution_list </b> - used in Step 8.
  - List of resolutions used in creating clusters
  - The first argument (0.1) is the starting value of the sequence.
  - The second argument (2) is the ending value of the sequence.
  - The third argument (0.1) specifies the increment (or step size) between each value in the sequence.
 
- <b> custom_palette_org </b>  - used in creating clustering plots

In [ ]:

umap_clusters <- TRUE
resolution_list <- seq(0.1, 2, 0.1)

custom_palette_org <- c("#E64B35B2", "#4DBBD5B2", "#00A087B2", "#3C5488B2", "#F39B7FB2", "#8491B4B2", "#91D1C2B2", "#DC0000B2",
                    "#7E6148B2", "#B09C85B2", "#FF410DCC", "#6EE2FFCC", "#F7C530CC", "#95CC5ECC", "#D0DFE6CC", "#F79D1ECC",
                    "#748AA6CC", "#00468BB2", "#ED0000B2", "#42B540B2", "#0099B4B2", "#925E9FB2", "#FDAF91B2", "#AD002AB2",
                    "#ADB6B6B2", "#1B1919B2", "#FED439CC", "#709AE1CC", "#8A9197CC", "#D2AF81CC", "#FD7446CC", "#D5E4A2CC",
                    "#197EC0CC", "#F05C3BCC", "#46732ECC", "#71D0F5CC", "#370335CC", "#075149CC", "#C80813CC", "#91331FCC",
                    "#1A9993CC", "#FD8CC1CC", "#FAFD7CCC", "#82491ECC", "#24325FCC", "#B7E4F9CC", "#FB6467CC", "#526E2DCC",
                    "#FF6F00B2", "#C71000B2", "#008EA0B2", "#8A4198B2", "#5A9599B2", "#CC0C00B2", "#5C88DAB2", "#84BD00B2",
                    "#FFCD00B2", "#7C878EB2", "#171717B2", "#7D0226B2", "#300049B2", "#165459B2", "#3F2327B2", "#0B1948B2",
                    "#E71012B2", "#555555B2", "#193006B2", "#A8450CB2", "#D51317B2", "#F39200B2", "#EFD500B2", "#95C11FB2",
                    "#007B3DB2", "#31B7BCB2", "#0094CDB2", "#164194B2", "#6F286AB2", "#706F6FB2", "#F9CA24CC", "#F0932BCC",
                    "#EB4D4BCC", "#6AB04CCC", "#C7ECEECC", "#22A6B3CC", "#BE2EDDCC", "#4834D4CC", "#130F40CC", "#535C68CC")

In [ ]:
# We need to ask the users what value for 'dims' to use since they need to see the elbow plot first. Only If we're generating plots based on user data
if(umap_clusters)
{ 
  seu_obj_merged <- RunUMAP(seu_obj_merged, reduction = "pca", dims = 1:dims)
}

seu_obj_merged <- FindNeighbors(seu_obj_merged, reduction = "pca", dims = 1:dims)

In [ ]:
source("./08_Clustering.R")
seu_obj_merged <- perform_clustering_and_marker_genes(seu_obj_merged, output_folder, resolution_list, umap_clusters, dims, 
                                                      custom_palette_org, name, alpha, timepoints, levels)

source("./08_02_Cluster_tree.R")
draw_cluster_tree(seu_obj_merged, output_folder, name)

### Step 10: Find Marker Genes

Identifies which genes are highly and specifically expressed within a cluster.
It helps identifying which seed tissue is a cluster is related to.

- res - chosen resolution 

In [ ]:
res <- 0.2

In [ ]:
find_marker_genes(seu_obj_merged, output_folder, name, res, timepoints)

### Step 11: Save to RDS file 
You may change the filename

In [ ]:
# Save Seurat Object into a RDS file
saveRDS(seu_obj_merged, file = paste0(output_folder, "seurat_object.rds"))
